In [20]:
import sys
print(sys.executable)


c:\own projects\ArchiveLLM\venv\Scripts\python.exe


In [21]:
import os
import openai
import json
from dotenv import load_dotenv                          # damit api-Keys sicher aus der .env geladen werden


load_dotenv()                                           #umgebungsvariabeln laden (openai api key)


openai.api_key = os.getenv("OPENAI_API_KEY")                #bei openai anmelden      

#hiernach ist alles insofern bereit, das der api key genutzt werden kann
#wichtig: 1. Api key MUSS immer gesetzt werden
#        2. .env muss im gleichenb Verzeichnis wie das Skript liegen
#        3. .env muss den api key beinhalten (OPENAI_API_KEY=...)             |  BITTE DEN API KEY IN DIE GITIGNORE DATEI EINTRAGEN, DAMIT ER NICHT GETRACKT WIRD
#                                                                             | bei nichteinhalung wird der key evtl von openai als geleaket angezeigt und gesperrt!

In [22]:
import pandas as pd                                                             #pandas dient als tool für tabellenverarbeitung

df = pd.read_csv("../../output/alle_fragen_gesamt.csv")         #csv datei einlesen, die alle fragen enthält


top_themen = df["thema"].value_counts().head(3).index.tolist()              #top 3 Themen ermitteln

# Pro Thema: Aufgaben sammeln
themen_aufgaben = {
    thema: df[df["thema"] == thema]["frage"].tolist()        #alle fragen zu den top 3 themen(von top_themen)  in einer liste speichern  
    for thema in top_themen
}


# jetzt haben wir eine Liste mit den 3 häufigsten Themen + den originalfragen dazu 


In [ ]:
def generate_prompt(thema, fragen_liste):                           #nimmt them +zugehörige fragen als input
    #eindeutige Fragen, maximal 5, schön formatiert 
    joined = "\n".join(dict.fromkeys(frage.strip() for frage in fragen_liste[:5]))                       #nimmt die ersten 5 Fragen und entfernt Duplikate
    return f"""Hier sind fünf echte Prüfungsfragen zum Thema "{thema}":                     
{joined}

Erstelle daraus drei neue, inhaltlich vergleichbare Übungsfragen im selben Stil."""


from openai import OpenAI           

client = OpenAI()            #client wird erstellt um  api zu nutzen | bzw damit die api angesprochen werden kann

def call_gpt(prompt):                                            #nimmt den prompt als input  , sendet also den prompt an die api und gibt die antwort zurück
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",                                       #ANPASSBARS MODELL, ich nahm 3.5, aber 4 wäre auch möglich |je nach bedarf (aber kostenintensiver)
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,                                              #je nach bedarf anpasssbar, genauere erklärung der conf score im repo readme
    )
    return response.choices[0].message.content            #gibt die antwort zurück, die von der api kommt 



# gesamte logik: erstellt auf Basis Fragen einen Prompt und schickt ihn an GPT und gibt passende Übungsaufgaben zurück

In [24]:
results = {}                                                    #dictionary für  ergebnisse

for thema, fragen in themen_aufgaben.items():         #loop über alle themen 
    prompt = generate_prompt(thema, fragen)             #prompt wird erstellt== indem die themen und fragen an die generate_prompt funktion übergeben werden
    output = call_gpt(prompt)                      #output wird erstellt== indem der prompt an die call_gpt funktion übergeben wird 
    
    
    results[thema] = {                  #speichert die ergebnisse von result in first line 
        "prompt": prompt,
        "generated_tasks": output
    }

# speichern
os.makedirs("generated_tasks", exist_ok=True)   #erstellt den ordner, falls er nicht existiert
with open("generated_tasks/aufgaben_llm.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False) #speichert die ergebnisse in einer json datei, die im ordner generated_tasks gespeichert wird (ArchiveLLM\llm_generator\generated_tasks)


# Für jedes thema wird ein passender Prompt erstellt/iteriert, an gpt geschickt und das Ergebnis als JSON-Datei gespeichert


In [26]:
from fpdf import FPDF
import json

with open("generated_tasks/aufgaben_llm.json", "r", encoding="utf-8") as f:
    data = json.load(f)

pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", size=12)

pdf.set_title("Generierte Prüfungsaufgaben")

for thema, inhalt in data.items():
    pdf.set_font("Arial", 'B', size=14)
    pdf.cell(0, 10, f"Thema: {thema}", ln=True)

    pdf.set_font("Arial", '', size=12)
    original_fragen = "\n".join(inhalt["prompt"].split("\n")[1:6])
    pdf.multi_cell(0, 10, f"Originalaufgabe:\n{original_fragen}\n", align="L")
    pdf.multi_cell(0, 10, f"Generierte Aufgaben:\n{inhalt['generated_tasks']}\n\n", align="L")

output_path = "generated_tasks/aufgaben_llm_export.pdf"
pdf.output(output_path)

print(f"Export abgeschlossen: {output_path}")


Export abgeschlossen: generated_tasks/aufgaben_llm_export.pdf
